In [12]:
import numpy as np
import nltk
import string
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_word = stopwords.words('english')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
icd2name = {}
with open("ICD9_descriptions",'r') as f:
    for line in f.readlines():
        line = line.strip()
        id = line.split("\t")[0]
        name = line.split("\t")[1]
        if "." not in id:
            icd2name[id]=name

In [5]:
label_to_ix=np.load('label_to_ix.npy',allow_pickle=True).item()

In [67]:
def token(seq):
    seq = seq.replace("&apos;",'-')
    seq_ = ''
    kong = True
    for i in seq.lower():
        if i not in string.punctuation:
            seq_+=i
        else:
            seq_+=' '
    tokens = []
    for word in seq_.split(" "):
        tokens.append(word)
        '''
        if word not in stop_word:
            tokens.append(word)
        '''
    return tokens

token("Procedures and interventions, Not Elsewhere Classified")

['procedures', 'and', 'interventions', '', 'not', 'elsewhere', 'classified']

In [68]:
not_find = []
label_id2name = {}  #为344个标签索引
for i,label in enumerate(label_to_ix.keys()):
    id = label.split('_')[1]
    try:
        name = icd2name[id]
        name = token(name)
        label_id2name[i] = name
        # print(i,id,name)
    except:
        label_id2name[i] = "None"
        not_find.append(id)
        print(id,"not find !!")
print(not_find)

209 not find !!
539 not find !!
['209', '539']


In [26]:
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('../../emb/PubMed-and-PMC-w2v.bin', binary=True)


In [69]:
label2vec = []
for i in range(344):
    if label_id2name[i] != 'None':
        # print(label_id2name[i])
        vec = np.zeros(200)
        for word in label_id2name[i]:
            if word == '':
                continue
            try:
                vec+=word_vectors.word_vec(word)
            except:
                print(word,label_id2name[i],i)
                vec+=np.random.uniform(-0.01, 0.01, 200).astype("float32")
    else:
        print(i)
        vec = np.random.uniform(-0.01, 0.01, 200).astype("float32")
    label2vec.append(vec)

ckd ['chronic', 'kidney', 'disease', '', 'ckd', ''] 38
parietoalveolar ['other', 'alveolar', 'and', 'parietoalveolar', 'pneumonopathy'] 158
250
338


In [72]:
label2vec = np.array(label2vec)
label2vec.shape

(344, 200)

In [73]:
np.save('label_embedding',label2vec)